In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('key-value_rdd_op_joins')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 04:06:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Operations
- `groupByKey`: 이미 그룹이 지어진 상태에서 진행
    - 그룹핑 후에 특성 Transformations같은 연산
    - key값이 있는 상태에서 시작

- `groupBy()` :그룹을 생성하기 위함
    - `RDD.grouBy(numPartitions=None, partitionFunc=<function portable_hash>)`
    - 함수에 의해서 그룹이 생기는 연산

In [2]:
rdd = sc.parallelize([
    ('짜장면',15),
    ('짬뽕',10),
    ('짜장면',5)
])

In [4]:
# 항상 제일 앞에 있는 것이 Key임.
g_rdd = rdd.groupByKey()
g_rdd.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x7f8130e296a0>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x7f8130581640>)]

In [10]:
for i,j in g_rdd.collect():
    print(i,len(list(j)))

짜장면 2
짬뽕 1


In [9]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [11]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

In [12]:
# groupBy 사용하기
rdd = sc.parallelize([
    
    'c','c++','c#','Python','Java','JavaScript'
])

In [13]:
rdd

ParallelCollectionRDD[13] at readRDDFromFile at PythonRDD.scala:274

In [14]:
# groupBy는 그룹핑 할 키에 대한 정의를 개발자가 직접 해준다.
grouped = rdd.groupBy(lambda x : x[0])
grouped.glom().collect()

[[('c', <pyspark.resultiterable.ResultIterable at 0x7f815c4819a0>),
  ('P', <pyspark.resultiterable.ResultIterable at 0x7f81304118b0>),
  ('J', <pyspark.resultiterable.ResultIterable at 0x7f8130411bb0>)]]

In [15]:
grouped.mapValues(list).collect()

[('c', ['c', 'c++', 'c#']), ('P', ['Python']), ('J', ['Java', 'JavaScript'])]

In [16]:
grouped.getNumPartitions()

1

In [17]:
g_rdd.getNumPartitions()

1

In [18]:
# groupByKey는 K-V RDD를 사용할 때 Key가 알아서 그룹핑의 기준이 된다.

x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

y = x.groupByKey()

In [19]:
y.collect()

[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f8130411d60>),
 ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f813005a8e0>),
 ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f813005a2b0>)]

In [20]:
y.glom().collect()

[[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f81302aac70>),
  ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f81302aad00>),
  ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f81302aad60>)],
 [],
 []]

# Join

In [21]:
# Inner Joins : 서로 간에 존재하는 키만 합쳐줍니다. (only 교집합)
rdd1 = sc.parallelize([
    ('foo',1),
    ('goo',2),
    ('hoo',3)
])

rdd2 = sc.parallelize([
    ('foo',1),
    ('goo',2),
    ('goo',10),
    ('moo',6)
])

rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

**Outer Join**
- 기준이 되는 한쪽에 데이터가 있고, 다른쪽에는 데이터가 없는 경우
    - 설정한 기준에 따라ㅓㅅ 기준에 맞는 데이터가 항상 남아있는다.
- `leftOuterJoin`: 왼쪽에 있는 rdd가 기준이 됩니다.(함수를 호출하는 쪽)
- `rightOuterJoin`: 오른쪽에 있는 rdd가 기준이 됩니다.(함수에 매개변수로 들어가는 쪽)

In [22]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [23]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [24]:
sc.stop()

데이터 관리는 RDD로 하지말고 SQL이나 그런걸로 하는게 최고.
이렇게 느리게 하면 큰일남.
지금은 그냥 개념 알려고 하는 것.